In [1]:
from typing import List, Callable
from interactive_visualization.animation_utils import animate_list
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import grad


# Методы


In [2]:
# malkovsky: лайк за typing
def ternary_search(fun: Callable[[float], float],
                   left: float,
                   right: float,
                   eps: float) -> (float, List[float]):
    """
    Вычисляет минимум функции fun на отрезке [left, right]

    args:
        fun: функция
        left, right: границы отрезка, где искать корень
        eps: погрешность для остановки

    Returns:
        (ответ, последовательность приближений)
    """
    assert (left <= right)
    assert (eps > 0)
    states = []
    while right - left >= eps:
        change = (right - left) / 3
        x1, x2 = left + change, left + 2 * change
        if fun(x1) >= fun(x2):
            states.append(left)
            left = x1
        else:
            states.append(right)
            right = x2
    return (right + left) / 2, states

In [3]:
def root_bisection(fun: Callable[[float], float],
                   left: float,
                   right: float,
                   eps: float) -> (float, List[float]):
    """
    Находит корень функции fun на отрезке [left, right]

    args:
        fun: функция
        left, right: границы отрезка, где искать корень
        eps: точность для нахождения корня

    Returns:
        (ответ, последовательность приближений)
    """
    assert (left <= right)
    assert (eps > 0)
    assert (fun(left) * fun(right) <= 0)
    if fun(left) == 0:
        return left, [left]
    if fun(right) == 0:
        return right, [right]
    states = []
    mid = left
    while right - left >= eps:
        mid = (left + right) / 2
        if fun(mid) * fun(right) > 0:
            states.append(right)
            right = mid
        else:
            states.append(left)
            left = mid
    states.append(mid)
    return mid, states

In [4]:
def method_newton(fun: Callable[[float], float],
                  df: Callable[[float], float],
                  x: float,
                  eps: float = 0.1,
                  iters: int = 100) -> (float, List[float]):
    """
    Находит корень функции fun на за iters шагов (или раньше, если получилась нужная точность)

    args:
        fun: функция
        dfun = дифференциал fun
        x: начальное приближение
        eps: точность для нахождения корня
        iters: число итераций

    Returns:
        (ответ, последовательность приближений)
    """
    assert (eps > 0)
    states = []
    for _ in range(iters):
        dfu = df(x) if df(x) != 0 else -fun(x)  # если 0 - двигаем x на 1 вправо
        x_new = x - fun(x) / dfu
        states.append(x)
        if abs(x - x_new) < eps:
            x = x_new
            break
        x = x_new
    states.append(x)
    return x, states

# Вспомогательные функции

In [5]:
def do_picture(f, points, lines, left, right):
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.tick_params(
        labelsize = 15,
        labelcolor = 'b',
    )
    y_min = min(f(left), f(right), -10)
    y_max = max(f(left), f(right), 10)
    delta = (right - left) / 100
    r = np.arange(left, right + delta, delta)

    ax.plot(r, [f(x) for x in r])
    ax.plot([left, left], [y_min, y_max], color='black')
    ax.plot([right, right], [y_min, y_max], color='black')
    ax.plot([left, right], [0, 0], color='black', linestyle='--')

    ax.scatter(points, list(map(f, points)), color='black')
    for pair in lines:
        ax.plot(pair, list(map(f, pair)), color='grey', linestyle='--')

    plt.close(fig)
    return fig

In [6]:
def draw(fun, states, left, right, metod):
    def update_states_for_bisection():
        points = []
        lines = []
        l = left
        r = right
        for st in states:
            m = (l + r) / 2
            points.append([l,m,r])
            if st == l:
                lines.append([[l,m]])
                l = m
            else:
                lines.append([[r,m]])
                r = m
        return points, lines

    def update_states_for_newton():
        points = []
        lines = []
        for i, st in enumerate(states[1:]):
            points.append([states[i], st])
            lines.append([[states[i], st]])
        return points, lines

    def update_states_for_roots_extrs():
        return states, [[[]], [[]]]

    def update_states_for_ternary():
        points = []
        lines = []
        l = left
        r = right
        for st in states:
            ch = (r - l) / 3
            points.append([l, l + ch, l + 2 * ch, r])
            if st - l < r - st:
                lines.append([[l, l + ch]])
                l = l + ch
            else:
                lines.append([[r, r - ch]])
                r = r - ch
        return points, lines


    def update_states():
        if metod == "bisection":
            return update_states_for_bisection()
        elif metod == "newton":
            return update_states_for_newton()
        elif metod == "roots_extrs":
            return update_states_for_roots_extrs()
        elif metod == "ternary":
            return update_states_for_ternary()
        else:
            raise Exception(f"нет рисования для метода {metod}")

    if len(states) == 0:
        print("нет состояний")
    else:
        points, lines = update_states()
        pictures = []
        for i, p in enumerate(points):
            pictures.append(do_picture(fun, p, lines[i], left, right))
        animate_list(pictures)

In [7]:
def dfun(fun, is_polinomial=False):
    if is_polinomial:
        return np.polyder(fun)
    else:
        df = grad(fun)
        return lambda x: df(np.array([x], float))[0]
        # h = 1e-8
        # return lambda x: (fun(x+h) - fun(x-h))/(2 * h)  # symmetric difference, working only for 1 dif

In [8]:
def find_left_right(fun):
    left, right = -1, 1
    while fun(left) > 0:
        left *= 2
    while fun(right) < 0:
        right *= 2
    return left, right

In [9]:
EPS = 0.05

# Task 1

1) Вычислить $\sqrt[k]{a}$ ($k \in \mathbb{N}, a \in \mathbb{R}$)

1б С помощью метода бисекции.

2б C помощью метода Ньютона.

Если $k\mod2=0$, то ответ больше 0 (корень чётной степени) - ограничение слева. Для произвольной ситуации: возьмём слева и справа максимум из $a$ по модулю (в случае, $a > 1$ корень сжимает к 0) и 1 (для $a < 1$ корень прижимает к 1). Сделали границу для объединения случаев

Начальное приближение можно взять 0. Корень либо прижимается снизу к 1 по модулю (около 0), либо стремится к 1 сверху (если не учитывать знак 0 - оптимальный)

In [10]:
def task1_bisection(k: int, a: float, visual=False):
    fun = lambda x: pow(x, k) - a
    gran = max(1., abs(a))
    left, right = (-gran if k % 2 != 0 else 0), gran
    ans, states = root_bisection(fun, left, right, EPS)
    if visual:
        draw(fun, states, left, right, "bisection")
    return ans if k % 2 != 0 else abs(ans)

In [11]:
def task1_newton(k: int, a: float, visual=False):
    fun = lambda x: pow(x, k) - a
    ans, states = method_newton(fun, dfun(fun), 0, EPS)
    if visual:
        gran = max(1., abs(a))
        left, right = (-gran if k % 2 != 0 else 0), gran
        draw(fun, states, left, right, "newton")
    return ans if k % 2 != 0 else abs(ans)

In [12]:
print(task1_bisection(2, 25, visual=True))
print(task1_newton(2, 25, visual=True))

interactive(children=(IntSlider(value=0, description='step', max=9), Output()), _dom_classes=('widget-interact…

5.029296875


interactive(children=(IntSlider(value=0, description='step', max=5), Output()), _dom_classes=('widget-interact…

5.000023178253949


In [13]:
print(task1_bisection(11, 1234, visual=True))
print(task1_newton(11, 1234, visual=True))


interactive(children=(IntSlider(value=0, description='step', max=16), Output()), _dom_classes=('widget-interac…

1.92059326171875


interactive(children=(IntSlider(value=0, description='step', max=46), Output()), _dom_classes=('widget-interac…

1.910334201101316


In [14]:
print(task1_bisection(5, 991324, visual=True))
print(task1_newton(5, 991324, visual=True))

interactive(children=(IntSlider(value=0, description='step', max=26), Output()), _dom_classes=('widget-interac…

15.805910229682922


interactive(children=(IntSlider(value=0, description='step', max=46), Output()), _dom_classes=('widget-interac…

15.821509366405662


In [15]:
print(task1_bisection(3, -125, visual=True))
print(task1_newton(3, -125, visual=True))

interactive(children=(IntSlider(value=0, description='step', max=13), Output()), _dom_classes=('widget-interac…

-4.974365234375


interactive(children=(IntSlider(value=0, description='step', max=9), Output()), _dom_classes=('widget-interact…

-5.000019538894343


In [16]:
print(task1_bisection(1, 8, visual=True))
print(task1_newton(1, 8, visual=True))

interactive(children=(IntSlider(value=0, description='step', max=0), Output()), _dom_classes=('widget-interact…

8


interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

8.0


# Task 2

2) Дан многочлен $P$ степени не больше $5$ и отрезок $[L; R]$, пара.

2б Локализовать корни $P$, т.е. найти непересекающиеся отрезки $[L_1; R_1] \ldots [L_m; R_m]$ такие, что $P(L_i) P (R_i) < 0$.

1б Для каждого $[L_i; R_i]$ найти соответствующий корень.

2б Найти глобальный минимум $P$ на отрезке $[L; R]$.

Используем честные производные для многочлена.

На каждой итерации 0 переходят в экстремумы, смотрим на всё только в нужном промежутке

In [17]:
def task2(polynom: np.poly1d, left: float, right: float):
    def find_correct_lrs(extrs):
        lrs = []
        last = extrs[0]
        for ext in extrs:
            if polynom(last) * polynom(ext) <= 0 \
                    or abs(polynom(ext)) < EPS:  # считаем, что кратный корень
                lrs.append((last, ext))
                last = ext
        return lrs

    deg = len(polynom)
    eps = 0.0001
    dfuns = [polynom]
    for _ in range(deg-1):
        dfuns.append(dfun(dfuns[-1], is_polinomial=True))
    lrs = [(left, right)]
    extrs = []
    roots = []
    for i, f in enumerate(reversed(dfuns)):
        extrs = roots
        roots = []
        for (l, r) in lrs:
            if f(l) * f(r) <= 0:  # иначе игнорируем, нас интересуют экстремумы только в [L; R]
                root, _ = root_bisection(f, l, r, eps)
                roots.append(root)
            elif abs(f(r)) < EPS:  # для кратных корней
                roots.append(r)
        if i != deg - 1:  # не обновляем lrs в последний раз
            new_lrs = [left] + roots + [right]
            lrs = []
            for j, ext in enumerate(new_lrs[1:]):
                lrs.append((new_lrs[j], ext))
    extrs = [left] + extrs + [right]
    minimum = min(map(lambda x: f(x), extrs))
    draw(polynom, [roots, extrs], left, right, "roots_extrs")
    return {"lrs" : find_correct_lrs(extrs), "roots" : roots, "minimum" : minimum}

In [18]:
# x**2 + x - 2
fun = np.poly1d([1, 1, -2])
task2(fun, -5, 5)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [(-5, -0.4999542236328125), (-0.4999542236328125, 5)],
 'roots': [-1.9999465940054506, 0.9999951452482492],
 'minimum': -2.249999997904524}

In [19]:
# x**3 + 5 x**2 - 10
fun = np.poly1d([1, 5, 0, -10])
task2(fun, -5, 5)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [(-5, -3.3333714806940407),
  (-3.3333714806940407, -3.1788949854671955e-05),
  (-3.1788949854671955e-05, 5)],
 'roots': [-4.507915316031337, -1.7556605447471885, 1.2634802724636973],
 'minimum': -10}

In [20]:
task2(fun, 0, 6)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [(0, 6)], 'roots': [1.263519287109375], 'minimum': -10}

In [21]:
task2(fun, -5, 0)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [(-5, -3.3333714806940407), (-3.3333714806940407, 0)],
 'roots': [-4.507915316031337, -1.7556454986437409],
 'minimum': -10}

In [22]:
task2(fun, -10, -5)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [], 'roots': [], 'minimum': -510}

In [23]:
fun2 = np.poly1d([3,10,0])
task2(fun2, -5, 0)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [(-5, -1.6666412353515625), (-1.6666412353515625, 0)],
 'roots': [-3.3333714806940407, 0],
 'minimum': -8.333333331393078}

In [24]:
# 3 x**5 - 6 x**4 + 5 x**3 + 4 x**2 - 14 x + 8
fun = np.poly1d([3, -6, 5, 4, -14, 8])
task2(fun, -1.5, 1.5)

interactive(children=(IntSlider(value=0, description='step', max=1), Output()), _dom_classes=('widget-interact…

{'lrs': [(-1.5, -0.7059941310435534),
  (-0.7059941310435534, 0.9999693660065532)],
 'roots': [-1.2273512561798725, 0.9999693660065532],
 'minimum': -32.03125}

# Task 3

3) При $a, b, c > 0$ найти минимум функции $e^{ax} + e^{−bx} + c(x − d)^2$

1б С использованием метода бисекции.

2б C использованием метода Ньютона.

1б С использованием тернарного поиска.

$(e^{ax} + e^{−bx} + c(x − d)^2)` = a e^{ax} - b e^{-bx} - 2c(d - x)$

Линейная комбинация с положительными коэффициентами выпуклых функций выпукла. Экспонента и парабола выпуклы. Значит наша функция тоже выпукла.

Производная выпуклой функции одной переменной - неубывающая функция. Наша функция на бесконечностях больше, чем около нуля за счёт экспоненты. Тогда у неё есть единственный экстремум, он соответствует нулю производной.

Тогда будем искать 0 производной. Придумаем границы и начальное приближение.

Подберём границы, чтобы они соответствовали условиям методов.

Для бисекции - подбор по производной функции, для тернарного поиска границы те же - мы знаем, что корень функциии лежит между этими границами, выпуклость есть

Для Ньютона возьмём середину из этих границ (не самое худшее приближение)

In [25]:
def task3_bisection(a: float, b: float, c: float, d: float, eps=EPS, visual=False):
    assert (a > 0 and b > 0 and c > 0)
    fun = lambda x: np.exp(a * x) + np.exp(-b * x) + c * np.power((x - d), 2)
    df = dfun(fun)
    left, right = find_left_right(df)
    ans, states = root_bisection(df, left, right, eps)
    if visual:
        draw(fun, states, left, right, "bisection")
        draw(df, states, left, right, "newton")
    return fun(ans)

In [26]:
def task3_newton(a: float, b: float, c: float, d: float, eps=EPS, visual=False):
    assert (a > 0 and b > 0 and c > 0)
    # fun = lambda x: exp(a * x) + exp(-b * x) + c * pow((x - d), 2)
    fun = lambda x: np.exp(a * x) + np.exp(-b * x) + c * np.power((x - d), 2)
    df = dfun(fun)
    left, right = find_left_right(df)
    ans, states = method_newton(df, dfun(df), (left + right) / 2, eps, iters=1000)
    if visual:
        draw(fun, states, left, right, "newton")
        draw(df, states, left, right, "newton")
    return fun(ans)

In [27]:
def task3_ternary(a: float, b: float, c: float, d: float, eps=EPS, visual=False):
    assert (a > 0 and b > 0 and c > 0)
    fun = lambda x: np.exp(a * x) + np.exp(-b * x) + c * np.power((x - d), 2)
    left, right = find_left_right(dfun(fun))
    ans, states = ternary_search(fun, left, right, eps)
    if visual:
        draw(fun, states, left, right, "ternary")
    return fun(ans)

In [28]:
eps = 0.00001

a, b, c, d = 1,2,3,4
task3_bisection(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=19), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=0, description='step', max=18), Output()), _dom_classes=('widget-interac…

18.652352027216903

In [29]:
task3_newton(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=4), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=0, description='step', max=4), Output()), _dom_classes=('widget-interact…

18.652352027148623

In [30]:
task3_ternary(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=32), Output()), _dom_classes=('widget-interac…

18.652352027149448

In [31]:
a, b, c, d = 5, 10, 23, 5
task3_bisection(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=18), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=0, description='step', max=17), Output()), _dom_classes=('widget-interac…

457.82394277270333

In [32]:
task3_newton(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=7), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=0, description='step', max=7), Output()), _dom_classes=('widget-interact…

457.82394276952766

In [33]:
task3_ternary(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=30), Output()), _dom_classes=('widget-interac…

457.82394276955245

In [34]:
a, b, c, d = 1, 1, 9, -4
task3_bisection(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=19), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=0, description='step', max=18), Output()), _dom_classes=('widget-interac…

29.080475686681318

In [35]:
task3_newton(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=4), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=0, description='step', max=4), Output()), _dom_classes=('widget-interact…

29.080475686125236

In [36]:
task3_ternary(a,b,c,d,eps=eps,visual=True)

interactive(children=(IntSlider(value=0, description='step', max=32), Output()), _dom_classes=('widget-interac…

29.080475686131575